In [1]:
from glob import glob
import os.path as op
import nibabel as nib
from shutil import copyfile

import pickle
import gzip

In [2]:
work_dir = '/home/data/nbc/external-datasets/ltd_dset/derivatives/fmriprep-work/'
deriv_dir = '/home/data/nbc/external-datasets/ltd_dset/derivatives/fmriprep/'
subs = ['ltd']

xform_rename = {'bold_reg_wf/bbreg_wf/fsl2itk_fwd/affine.txt': '_from-reference_to-T1w_mode-image_xfm.txt',
                'bold_hmc_wf/fsl2itk/mat2itk.txt': '_from-native_to-reference_mode-image_xfm.txt'}

In [3]:
for sub in subs:
    sub_in_dir = op.join(work_dir, 'single_subject_{0}_wf'.format(sub))
    task_dirs = glob(op.join(sub_in_dir, 'func_preproc_task_*_wf'))
    for task_dir in task_dirs:
        bb_wf_dir = op.join(task_dir, 'bold_bold_trans_wf')
        bf_dirs = sorted(glob(op.join(bb_wf_dir, '_bold_file_*')))
        for bf_dir in bf_dirs:
            # Collect partially preprocessed data
            bf_dir_list = bf_dir.split('..')
            idx = bf_dir_list.index('sub-{0}'.format(sub))
            sub_deriv_dir = op.join(deriv_dir, op.dirname('/'.join(bf_dir_list[idx:])))
            bf_filename = bf_dir_list[-1]
            in_file = op.join(bf_dir, 'merge/vol0000_xform-00000_merged.nii.gz')
            orig_fn_list = bf_filename.split('_')
            fn_list = orig_fn_list[:]
            fn_list.insert(-1, 'space-native')
            fn_list.insert(-1, 'desc-partialPreproc')
            out_file = op.join(sub_deriv_dir, '_'.join(fn_list))
            copyfile(in_file, out_file)
            
        # Collect native-to-T1w and T1w-to-MNI transforms
        out_func_dir = op.dirname(out_file)
        f = op.join(task_dir, 'bold_mni_trans_wf',
                    'bold_to_mni_transform/_inputs.pklz')
        with gzip.open(f, 'rb') as fo:
            data = pickle.load(fo)
        
        xform_rename2 = {}
        orig_fn_list = [fn for fn in orig_fn_list if 'echo' not in fn]
        orig_fn_list = orig_fn_list[:-1]
        for xform in xform_rename.keys():
            chosen = [xf for xf in data['transforms'] if xf.endswith(xform)]
            assert len(chosen) == 1
            chosen = chosen[0]
            xform_fn = '_'.join(orig_fn_list) + xform_rename[xform]
            xform_rename2[chosen] = op.join(out_func_dir, xform_fn)
                
        for in_xform in xform_rename2.keys():
            copyfile(in_xform, xform_rename2[in_xform])


In [4]:
# TODO: build miniworkflow to split, apply xforms, and merge